<a href="https://colab.research.google.com/github/Silfa2/nlpcc-ui-2025/blob/main/week5_6_task1_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Gabungan Minggu 5 & 6: Aplikasi LLM Interaktif

**Nama:** Silfa Wulandari

**NPM:** 2206090192

**Tugas 1: Aplikasi Obrolan LLM Sederhana**
------

## 🎯 Objektif Tugas 1
Membuat aplikasi obrolan dasar berbasis teks (command-line) yang interaktif menggunakan LLM (OpenAI API) untuk merespons input pengguna. Aplikasi ini akan didesain sebagai *multi-turn chat*, artinya ia akan mengingat konteks percakapan sebelumnya.

## 📚 Langkah Pengerjaan
Notebook ini akan memandu langkah-langkah pembuatan aplikasi chat:
1.  **Instalasi & Impor Library**: Menyiapkan semua paket Python yang dibutuhkan.
2.  **Pengaturan Kunci API**: Mengambil kunci API OpenAI dengan aman menggunakan Colab Secrets.
3.  **Inisialisasi Klien OpenAI**: Membuat objek klien untuk berinteraksi dengan API.
4.  **Logika Inti Aplikasi Chat**:
    *   Mendefinisikan fungsi untuk mengirim pesan dan menerima respons dari LLM.
    *   Mengelola riwayat percakapan (*chat history*).
    *   Membuat *loop* interaktif untuk pengguna.
5.  **Menjalankan Aplikasi Chat**: Demonstrasi cara menjalankan aplikasi.
6.  **Penjelasan & Contoh Interaksi**: Detail mengenai pilihan desain, model yang digunakan, dan contoh percakapan.



In [1]:
# Tahap 1.1: Instalasi Library OpenAI
# Kita gunakan '-q' (quiet) agar output instalasinya tidak terlalu ramai.
!pip install openai -q

# Tahap 1.2: Impor Library yang Dibutuhkan
import openai # Library utama untuk OpenAI
from google.colab import userdata # Untuk mengakses Colab Secrets (kunci API)
import os # Berguna untuk alternatif manajemen kunci API jika tidak di Colab
from IPython.display import display, Markdown # Untuk menampilkan output Markdown yang lebih cantik di Colab

print("✅ Library OpenAI telah terinstal dan library lain telah diimpor!")

✅ Library OpenAI telah terinstal dan library lain telah diimpor!


## 🔑 Tahap 2: Pengaturan Kunci API & Inisialisasi Klien OpenAI

Untuk berinteraksi dengan API OpenAI, kita memerlukan kunci API. Kunci ini bersifat rahasia dan tidak boleh dibagikan atau ditulis langsung dalam kode (*hardcode*). Kita akan menggunakan fitur **Colab Secrets** untuk menyimpannya dengan aman.


In [3]:
# Tahap 2.1: Mengambil Kunci API dari Colab Secrets
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        # Jika kunci ada tapi kosong
        display(Markdown("⚠️ **Peringatan:** Kunci API 'openai_key' ditemukan di Colab Secrets, tetapi nilainya kosong. Harap periksa kembali."))
        raise ValueError("Kunci API 'openai_key' kosong.")
    print("✅ Kunci API OpenAI berhasil diambil dari Colab Secrets.")

except userdata.SecretNotFoundError:
    display(Markdown("❌ **Error:** *Secret* 'openai_key' tidak ditemukan di Colab Secrets."))
    display(Markdown("Silakan ikuti langkah-langkah di atas untuk menambahkan kunci API Anda. Tanpa kunci API, aplikasi tidak akan berfungsi."))
    OPENAI_API_KEY = None # Set ke None agar langkah selanjutnya bisa mendeteksi
except ValueError as ve:
    # Ini akan menangkap error dari 'raise ValueError' di atas jika kunci kosong
    OPENAI_API_KEY = None
except Exception as e:
    display(Markdown(f"🚨 **Error tak terduga saat mengambil kunci API:** {str(e)}"))
    OPENAI_API_KEY = None

# Tahap 2.2: Inisialisasi Klien OpenAI
# Hanya lanjutkan jika kunci API berhasil didapatkan
if OPENAI_API_KEY:
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        print("🤖 Klien OpenAI berhasil diinisialisasi!")
        display(Markdown("🎉 **Selamat! Klien OpenAI siap digunakan.** Kita bisa lanjut ke tahap berikutnya."))
    except Exception as e:
        display(Markdown(f"🔥 **Error saat inisialisasi klien OpenAI:** {str(e)}"))
        display(Markdown("Pastikan kunci API Anda valid dan memiliki akses ke model yang akan digunakan."))
        client = None # Set client ke None jika inisialisasi gagal
else:
    display(Markdown("🔴 **Klien OpenAI tidak dapat diinisialisasi karena kunci API tidak tersedia atau bermasalah.**"))
    client = None

✅ Kunci API OpenAI berhasil diambil dari Colab Secrets.
🤖 Klien OpenAI berhasil diinisialisasi!


🎉 **Selamat! Klien OpenAI siap digunakan.** Kita bisa lanjut ke tahap berikutnya.

## 🧠 Tahap 3: Fungsi Inti Aplikasi Chat Multi-Turn

Sekarang kita akan membangun "otak" dari chatbot kita. Kita akan membuat sebuah fungsi yang canggih untuk menangani percakapan multi-putaran (***multi-turn***). Ini berarti chatbot kita akan mengingat apa yang sudah kita bicarakan sebelumnya, membuat interaksi terasa lebih alami dan cerdas!

Fungsi `chat_with_ai` akan melakukan hal berikut:
1.  Menerima input dari pengguna.
2.  Menambahkan input pengguna ke dalam riwayat percakapan.
3.  Mengirim seluruh riwayat percakapan ke model LLM OpenAI.
4.  Menerima balasan dari AI.
5.  Menambahkan balasan AI ke riwayat percakapan.
6.  Mengembalikan balasan AI untuk ditampilkan.

Kita juga akan membuat fungsi `display_chat_bubble` untuk menampilkan pesan dengan gaya gelembung obrolan agar lebih menarik secara visual.

In [4]:
# Tahap 3.1: Fungsi untuk menampilkan pesan dalam gelembung obrolan
def display_chat_bubble(role, message, model_name=None):
    """Menampilkan pesan dengan gaya gelembung obrolan di Colab."""
    if role == "user":
        bubble_style = "background-color: #E1F5FE; color: #01579B; border-radius: 15px 15px 0 15px; padding: 10px 15px; margin: 5px 50px 5px 5px; max-width: 70%; float: left; clear: both; box-shadow: 2px 2px 5px rgba(0,0,0,0.1);"
        align_container = "display: flex; justify-content: flex-start;"
        avatar = "👤"
        prefix = "Anda"
    elif role == "assistant":
        bubble_style = "background-color: #FCE4EC; color: #880E4F; border-radius: 15px 15px 15px 0; padding: 10px 15px; margin: 5px 5px 5px 50px; max-width: 70%; float: right; clear: both; box-shadow: -2px 2px 5px rgba(0,0,0,0.1);"
        align_container = "display: flex; justify-content: flex-end;"
        avatar = "🤖"
        prefix = f"AI ({model_name})" if model_name else "AI"
    else: # system message or others
        display(Markdown(f"_{message}_"))
        return

    # Membersihkan pesan dari karakter yang bisa mengganggu HTML
    message_cleaned = message.replace("<", "<").replace(">", ">")
    # Mengganti newline dengan <br> untuk HTML
    message_html = message_cleaned.replace('\n', '<br>')

    html_output = f"""
    <div style="{align_container}">
        <div style="{bubble_style}">
            <strong style="font-size: 0.9em;">{avatar} {prefix}:</strong><br>
            {message_html}
        </div>
    </div>
    <div style="clear: both;"></div>
    """
    display(Markdown(html_output))

# Tahap 3.2: Fungsi inti untuk berinteraksi dengan LLM (Multi-Turn)
def chat_with_ai(user_input, chat_history, model_name="gpt-3.5-turbo"):
    """
    Mengirim input pengguna dan riwayat percakapan ke OpenAI API,
    dan mengembalikan respons dari AI.
    """
    if not client:
        display(Markdown("🔥 **Error:** Klien OpenAI tidak diinisialisasi. Fungsi chat tidak dapat berjalan."))
        return "Maaf, saya tidak bisa merespons saat ini karena ada masalah koneksi.", chat_history

    # Tambahkan pesan pengguna ke riwayat
    chat_history.append({"role": "user", "content": user_input})

    # Menampilkan gelembung pesan pengguna
    # display_chat_bubble("user", user_input) # Kita akan panggil ini di loop utama

    try:
        # Kirim seluruh riwayat ke API
        response = client.chat.completions.create(
            model=model_name,
            messages=chat_history
        )
        ai_reply = response.choices[0].message.content

        # Tambahkan balasan AI ke riwayat
        chat_history.append({"role": "assistant", "content": ai_reply})

        # Menampilkan gelembung pesan AI
        # display_chat_bubble("assistant", ai_reply, model_name) # Kita akan panggil ini di loop utama

        return ai_reply, chat_history

    except openai.APIError as e:
        error_message = f"😥 Maaf, terjadi kesalahan API OpenAI: {e}"
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_message}</div>"))
        # Hapus pesan pengguna terakhir dari riwayat jika terjadi error, agar tidak dikirim ulang
        if chat_history and chat_history[-1]["role"] == "user":
            chat_history.pop()
        return error_message, chat_history
    except Exception as e:
        error_message = f"🤯 Aduh, ada error tak terduga: {e}"
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_message}</div>"))
        if chat_history and chat_history[-1]["role"] == "user":
            chat_history.pop()
        return error_message, chat_history

# Tes kecil untuk memastikan fungsi bisa di-define (tidak akan menghasilkan output chat)
if client:
    print("✅ Fungsi `display_chat_bubble` dan `chat_with_ai` berhasil didefinisikan.")
    print("   Kita akan menggunakannya di tahap berikutnya untuk membuat loop interaktif.")
else:
    print("⚠️ Fungsi didefinisikan, tetapi klien OpenAI tidak siap. Aplikasi chat mungkin tidak berfungsi.")

✅ Fungsi `display_chat_bubble` dan `chat_with_ai` berhasil didefinisikan.
   Kita akan menggunakannya di tahap berikutnya untuk membuat loop interaktif.


## 💬 Tahap 4: Loop Interaktif Aplikasi Chat

Inilah saatnya menghidupkan chatbot kita! Kode di bawah ini akan menjalankan *loop* utama aplikasi:
1.  **Inisialisasi Riwayat**: Kita mulai dengan riwayat percakapan kosong, namun kita akan menambahkan "pesan sistem" (*system message*) untuk memberi tahu AI bagaimana ia harus bersikap (misalnya, sebagai asisten yang membantu).
2.  **Pilihan Model**: Anda bisa memilih model LLM yang ingin digunakan.
3.  **Loop Utama**:
    *   Program akan meminta Anda mengetikkan pesan.
    *   Pesan Anda dan respons AI akan ditampilkan menggunakan `display_chat_bubble` yang sudah kita buat.
    *   Ketik `exit` atau `quit` untuk mengakhiri percakapan.
    *   Ada juga batasan sederhana pada panjang riwayat untuk mencegah penggunaan token yang berlebihan dalam sesi demo yang panjang.

In [6]:
# Tahap 4.1: Inisialisasi dan Menjalankan Loop Chat Interaktif

if not client:
    display(Markdown("🔴 **Tidak dapat menjalankan chat:** Klien OpenAI belum siap. Pastikan kunci API sudah benar dan klien terinisialisasi."))
else:
    display(Markdown("🚀 **Aplikasi Chat Siap Dimulai!** 🚀"))
    print("Ketik pesan Anda di bawah. Ketik 'exit' atau 'quit' untuk keluar.")
    print("-" * 50)

    # --- PENTING: PILIH MODEL ANDA DI SINI ---
    # Model yang direkomendasikan: "gpt-3.5-turbo" (cepat & hemat) atau "gpt-4o" (lebih canggih)
    # Pastikan kunci API Anda memiliki akses ke model yang dipilih.


    display(Markdown(f"ℹ️ *Menggunakan model: **{chosen_model}***"))

    # Inisialisasi riwayat percakapan dengan pesan sistem
    # Pesan sistem ini membantu mengatur perilaku AI.
    chat_history = [
        {"role": "system", "content": "Kamu adalah AsistenAI yang cerdas, ramah, dan sangat membantu. Kamu mengingat percakapan sebelumnya untuk memberikan respons yang relevan."}
    ]

    # Menampilkan pesan sistem (opsional, bisa di-comment jika tidak ingin terlihat)
    # display_chat_bubble("system", chat_history[0]["content"])

    MAX_HISTORY_LEN = 20 # Batasi riwayat menjadi N pesan terakhir (user + assistant) untuk efisiensi token

    while True:
        try:
            user_input = input("👤 Anda: ")
        except KeyboardInterrupt:
            display(Markdown("\n👋 Sesi chat dihentikan oleh pengguna. Sampai jumpa!"))
            break
        except EOFError: # Terjadi jika input stream ditutup (misalnya di beberapa environment non-interaktif)
            display(Markdown("\n👋 Input stream berakhir. Sampai jumpa!"))
            break


        if user_input.lower() in ['exit', 'quit']:
            display(Markdown("👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!"))
            break

        if not user_input.strip():
            display_chat_bubble("assistant", "Hmm, sepertinya Anda belum mengetik apa-apa. Ada yang bisa saya bantu?", chosen_model)
            continue

        # Tampilkan pesan pengguna terlebih dahulu
        display_chat_bubble("user", user_input)

        # Dapatkan respons dari AI
        ai_response, chat_history = chat_with_ai(user_input, chat_history, model_name=chosen_model)

        # Tampilkan respons AI
        display_chat_bubble("assistant", ai_response, chosen_model)

        # Manajemen riwayat sederhana: simpan N interaksi terakhir (pesan sistem + N pasangan user-assistant)
        # Ini untuk mencegah riwayat menjadi terlalu panjang dan mahal.
        # Setiap interaksi = 1 pesan user + 1 pesan assistant = 2 item di list
        # Jadi, jika MAX_HISTORY_LEN = 20, kita simpan 10 interaksi terakhir.
        if len(chat_history) > (MAX_HISTORY_LEN * 2 + 1): # +1 untuk pesan sistem
            display(Markdown(f"<i>(Riwayat percakapan dipangkas untuk efisiensi. Menyimpan {MAX_HISTORY_LEN} interaksi terakhir...)</i>"))
            # Simpan pesan sistem, lalu N interaksi terakhir
            chat_history = [chat_history[0]] + chat_history[-(MAX_HISTORY_LEN*2):]

    print("-" * 50)
    display(Markdown("🏁 **Sesi Chat Selesai.**"))

🚀 **Aplikasi Chat Siap Dimulai!** 🚀

Ketik pesan Anda di bawah. Ketik 'exit' atau 'quit' untuk keluar.
--------------------------------------------------


ℹ️ *Menggunakan model: **gpt-4o***

👤 Anda: hai



    <div style="display: flex; justify-content: flex-start;">
        <div style="background-color: #E1F5FE; color: #01579B; border-radius: 15px 15px 0 15px; padding: 10px 15px; margin: 5px 50px 5px 5px; max-width: 70%; float: left; clear: both; box-shadow: 2px 2px 5px rgba(0,0,0,0.1);">
            <strong style="font-size: 0.9em;">👤 Anda:</strong><br>
            hai
        </div>
    </div>
    <div style="clear: both;"></div>
    

...


    <div style="display: flex; justify-content: flex-end;">
        <div style="background-color: #FCE4EC; color: #880E4F; border-radius: 15px 15px 15px 0; padding: 10px 15px; margin: 5px 5px 5px 50px; max-width: 70%; float: right; clear: both; box-shadow: -2px 2px 5px rgba(0,0,0,0.1);">
            <strong style="font-size: 0.9em;">🤖 AI (gpt-4o):</strong><br>
            Jakarta merayakan hari ulang tahunnya pada tanggal 22 Juni. Tanggal ini dipilih berdasarkan penetapan Fatahillah atas nama Kerajaan Demak ketika berhasil merebut Sunda Kelapa dari Portugis pada 22 Juni 1527. Ada yang lain yang ingin Anda ketahui tentang Jakarta?
        </div>
    </div>
    <div style="clear: both;"></div>
    

👤 Anda: exit


👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!

--------------------------------------------------


🏁 **Sesi Chat Selesai.**

## ⚙️ Tahap 5: Penjelasan Aplikasi Chat & Contoh Interaksi

### Deskripsi Aplikasi
Aplikasi ini adalah sebuah chatbot sederhana berbasis teks yang berinteraksi dengan pengguna melalui *command-line interface* di Google Colab. Chatbot ini menggunakan API dari OpenAI untuk menghasilkan respons berdasarkan input pengguna.

### Fitur Utama & Pilihan Desain
1.  **Interaksi Programatik dengan LLM**:
    *   Aplikasi menggunakan *library* `openai` untuk mengirim permintaan ke model LLM.
    *   Model yang digunakan saat ini adalah **`gpt-4o`** (Anda bisa menggantinya di kode jika diperlukan, misalnya ke `gpt-3.5-turbo` untuk efisiensi atau model lain yang didukung kunci API Anda). Model `gpt-4o` dipilih karena kemampuannya yang canggih dalam memahami konteks dan menghasilkan respons yang alami.
2.  **Antarmuka Pengguna**:
    *   Interaksi dilakukan melalui input teks standar di sel Colab.
    *   Respons dari pengguna dan AI ditampilkan dengan gaya "gelembung obrolan" menggunakan HTML dan Markdown di output sel Colab untuk visualisasi yang lebih baik.
3.  **Fungsionalitas Multi-Turn**:
    *   Aplikasi ini dirancang sebagai *multi-turn chat*. Ini berarti ia **mengingat riwayat percakapan** sebelumnya.
    *   Setiap kali pengguna mengirim pesan baru, seluruh riwayat percakapan (termasuk pesan sistem awal yang mengatur perilaku AI) dikirimkan ke LLM. Hal ini memungkinkan AI memberikan respons yang lebih kontekstual dan relevan.
    *   Terdapat mekanisme sederhana untuk membatasi panjang riwayat yang dikirim ke API (`MAX_HISTORY_LEN`) untuk mengelola penggunaan token dan biaya, terutama dalam sesi demo yang panjang.
4.  **Pesan Sistem (System Message)**:
    *   Sebuah pesan sistem awal ("*Kamu adalah AsistenAI yang cerdas, ramah, dan sangat membantu. Kamu mengingat percakapan sebelumnya untuk memberikan respons yang relevan.*") digunakan untuk memberikan instruksi awal kepada LLM mengenai persona dan perilakunya.
5.  **Penanganan Input & Output**:
    *   Pengguna dapat mengetik `exit` atau `quit` untuk mengakhiri sesi chat.
    *   Input kosong dari pengguna akan ditangani dengan respons default dari AI.
    *   Ada penanganan error dasar untuk masalah API atau error tak terduga lainnya.

### Cara Menjalankan Aplikasi
1.  Pastikan Anda telah menambahkan kunci API OpenAI Anda ke Colab Secrets dengan nama `openai_key`.
2.  Jalankan semua sel kode dari Tahap 1 hingga Tahap 3.
3.  Jalankan sel kode di Tahap 4 yang berisi *loop* interaktif.
4.  Ketik pesan Anda pada *prompt* `👤 Anda:` dan tekan Enter.
5.  Untuk keluar, ketik `exit` atau `quit`.

### Contoh Interaksi
Berikut adalah contoh log percakapan dengan chatbot:

```text
🚀 Aplikasi Chat Siap Dimulai! 🚀
Ketik pesan Anda di bawah. Ketik 'exit' atau 'quit' untuk keluar.
--------------------------------------------------
ℹ️ Menggunakan model: *gpt-4o*

👤 Anda: hai
🤖 AI (gpt-4o): Hai! Apa kabar? Ada yang bisa saya bantu hari ini?

👤 Anda: apa ibukota indonesia
🤖 AI (gpt-4o): Ibu kota Indonesia adalah Jakarta. Ada yang lain yang ingin Anda ketahui?

👤 Anda: tanggal ulang tahun jakarta kapan ya
🤖 AI (gpt-4o): Jakarta merayakan hari ulang tahunnya pada tanggal 22 Juni. Tanggal ini dipilih berdasarkan penetapan Fatahillah atas nama Kerajaan Demak ketika berhasil merebut Sunda Kelapa dari Portugis pada 22 Juni 1527. Ada yang lain yang ingin Anda ketahui tentang Jakarta?

👤 Anda: exit
👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!
--------------------------------------------------
🏁 Sesi Chat Selesai.

# **Kesimpulan Tugas 1**

Penyelesaian Tugas 1 ini berhasil mendemonstrasikan kemampuan dasar dalam membangun aplikasi chat interaktif menggunakan Model Bahasa Skala Besar (LLM) dari OpenAI. Berikut adalah beberapa poin refleksi dan kesimpulan:

1.  **Keberhasilan Implementasi Inti**:
    *   Aplikasi berhasil mengimplementasikan interaksi programatik dengan API OpenAI, memungkinkan dialog antara pengguna dan LLM. Penggunaan model `gpt-4o` (atau model pilihan Anda) menunjukkan kemampuan untuk memanfaatkan model yang canggih untuk pemahaman bahasa alami dan generasi respons.
    *   Fitur *multi-turn chat*, yang dicapai dengan mengirimkan riwayat percakapan (`chat_history`) secara berkelanjutan, terbukti krusial. Tanpa ini, setiap interaksi akan terisolasi, mengurangi kemampuan chatbot untuk menjawab pertanyaan lanjutan atau merujuk pada konteks sebelumnya. Ini adalah perbedaan fundamental antara chatbot sederhana dan agen percakapan yang lebih cerdas.

2.  **Peningkatan Pengalaman Pengguna**:
    *   Visualisasi percakapan menggunakan "gelembung obrolan" (`display_chat_bubble`) secara signifikan meningkatkan keterbacaan dan pengalaman pengguna di lingkungan Colab, membuatnya lebih mirip dengan antarmuka chat modern.
    *   Pesan sistem awal memberikan arahan yang berguna bagi LLM untuk mengadopsi persona sebagai "AsistenAI yang cerdas, ramah, dan sangat membantu," yang berkontribusi pada kualitas dan gaya respons yang dihasilkan.

3.  **Keterbatasan dan Potensi Pengembangan Lebih Lanjut**:
    *   **Integrasi Alat (Tool Use)**: Chatbot saat ini murni berbasis teks. Untuk kemampuan yang lebih luas (misalnya, mengambil informasi *real-time*, melakukan kalkulasi), integrasi dengan alat eksternal melalui *function calling* (seperti yang akan dieksplorasi di Tugas Bonus) akan menjadi langkah berikutnya yang signifikan.
    *   **Evaluasi Kualitas**: Kualitas respons chatbot saat ini dinilai secara subjektif. Dalam pengembangan yang lebih serius, metrik evaluasi formal (misalnya, BLEU, ROUGE untuk tugas tertentu, atau evaluasi manusia terstruktur) akan diperlukan.

4.  **Pertimbangan Etis dan Biaya**:
    *   Penggunaan model LLM canggih seperti `gpt-4o` memberikan respons berkualitas tinggi, namun juga datang dengan biaya API yang lebih tinggi per token dibandingkan model seperti `gpt-3.5-turbo`. Pemilihan model harus selalu mempertimbangkan keseimbangan antara kualitas, biaya, dan latensi.
    *   Penting untuk selalu sadar akan potensi bias dalam data pelatihan LLM yang dapat muncul dalam respons, serta memastikan penggunaan AI yang bertanggung jawab.



## ✨ Tugas Bonus: Interaksi Multi-Modal - Memproses Gambar dan perlu disesuaikan untuk menyertakan gambar. Formatnya berbeda dari pesan teks murni. Biasanya melibatkan array `content` yang berisi Teks ✨

Untuk tugas bonus ini, saya memilih untuk mengimplementasikan fitur **Interaksi Multi-Modal** pada objek untuk teks dan objek untuk gambar (URL atau base64).

3.  **Pemilihan Model yang T aplikasi chat Tugas 1. Secara spesifik, saya akan meningkatkan chatbot agar dapat:

🎯 **Menerima inputepat:**
    *   Anda **harus** menggunakan model yang mendukung input visual, seperti `gpt-4o gambar dari pengguna bersama dengan kueri teks, dan meminta LLM untuk memproses serta merespons berdasarkan kedua jenis` atau `gpt-4-vision-preview` (jika menggunakan OpenAI). Model seperti `gpt-3.5-turbo input tersebut.**

**Alasan Pemilihan Fitur:**
1.  **Relevansi dengan Kemajuan AI Ter` tidak bisa memproses gambar. Pastikan `chosen_model` diatur dengan benar.

4.  **Penjelasan Markdown untuk Bonus:**
    *   Tambahkan bagian baru di *notebook* Tugas 1 khusus untuk menjelaskan fitur bonuskini**: Model multi-modal yang dapat memahami dan berinteraksi dengan berbagai jenis data (teks, gambar, ini, termasuk:
        *   Mengapa Anda memilih fitur ini.
        *   Bagaimana Anda mengintegrasikannya ( audio) merupakan salah satu ranah paling menarik dan berkembang pesat dalam AI generatif. Mengimplementasikan ini menunjukkan pemperubahan kode utama).
        *   Demonstrasi cara kerjanya (contoh input dengan URL gambar dan pertanyaanahaman terhadap kapabilitas model terkini.
2.  **Peningkatan Signifikan pada Aplikasi Chat**: Kemampuan untuk membahas, serta respons LLM).
        *   Tantangan yang mungkin dihadapi.

---

**Mari kita gambar secara langsung dalam percakapan membuka berbagai kasus penggunaan baru dan membuat interaksi jauh lebih kaya dan intuitif, mel mulai dengan modifikasi kode. Kita akan fokus pada notebook `week5_6_task1_chat.ipynb`.ampaui batasan teks murni. Pengguna bisa bertanya tentang objek dalam gambar, meminta deskripsi, atau bahkan**

**Tahap Bonus 1: Modifikasi Fungsi `chat_with_ai` dan Persiapan Input Gambar meminta interpretasi kreatif.
3.  **Demonstrasi Pemahaman API Multi-Modal**: Implementasi ini memerlukan pem**

Kita akan memperbarui fungsi `chat_with_ai` dan sedikit mengubah loop utama untuk menangani input gambar.

**ahaman tentang bagaimana mengirim data gambar (misalnya, sebagai URL atau base64) dan teks secara bersamaan ke *Di notebook `week5_6_task1_chat.ipynb` Anda:**

1.  **Pastikan Modelendpoint* API LLM yang mendukung multi-modal (seperti `gpt-4o` atau model Vision lainnya).
4. yang Dipilih Mendukung Vision:**
    Di sel kode Tahap 4 (Loop Interaktif Aplikasi Chat), pastikan  **Potensi Dampak yang Jelas**: Manfaat fitur ini mudah didemonstrasikan dan dipahami. Misalnya `chosen_model` diatur ke model yang mendukung visi. `gpt-4o` adalah pilihan yang baik.
    ```, pengguna mengunggah gambar kucing dan bertanya, "Jenis kucing apa ini dan apa yang sedang dilakukannya?" LLM kemudianpython
    # chosen_model = "gpt-3.5-turbo" # GANTI INI
    chosen_model = "gpt-4o" # Model ini mendukung teks dan gambar
    ```

2.  **Modifikasi akan merespons berdasarkan analisis gambar dan teks.

Fitur ini akan diintegrasikan ke dalam loop chat yang sudah ada, Fungsi `chat_with_ai` (dari Tahap 3 di Tugas 1):**
    Ganti dengan modifikasi pada cara pengguna memberikan input dan bagaimana pesan dikonstruksi untuk dikirim ke LLM. Model yang akan definisi fungsi `chat_with_ai` Anda dengan yang berikut. Perhatikan perubahan pada parameter dan cara `messages` dibuat ditargetkan untuk fitur ini adalah `gpt-4o` karena dukungannya yang kuat untuk input multi-modal.

In [7]:
# Tahap Bonus A.1: Fungsi untuk Encoding Gambar ke Base64
import base64
import mimetypes # Untuk mendapatkan tipe MIME gambar

def image_to_base64(image_bytes, mime_type=None):
    """Mengonversi byte gambar ke string data URI base64."""
    if not mime_type:
        # Jika tipe MIME tidak disediakan, coba tebak (ini mungkin tidak selalu akurat)
        # Untuk penggunaan yang lebih robust, sebaiknya ketahui tipe MIME dari file asli
        # Namun, untuk banyak format umum (jpg, png), ini seringkali cukup.
        # Kita akan asumsikan PNG atau JPEG jika tidak diketahui, tapi ini bisa disempurnakan.
        # Untuk Colab files.upload(), kita tidak langsung dapat nama file asli di sini.
        # Jadi, kita akan mengandalkan input mime_type atau default ke format umum.
        # Jika Anda tahu nama file, Anda bisa menggunakan:
        # mime_type, _ = mimetypes.guess_type(file_path)
        # Jika tidak, kita bisa coba default ke 'image/png' atau 'image/jpeg'
        # atau biarkan API LLM yang mencoba menentukannya jika format base64 saja cukup.
        # Untuk OpenAI, menyertakan tipe MIME di data URI adalah praktik terbaik.
        # Kita akan coba default ke png jika tidak ada.
        mime_type = "image/png" # Default jika tidak ada info

    base64_encoded_data = base64.b64encode(image_bytes).decode('utf-8')
    return f"data:{mime_type};base64,{base64_encoded_data}"

print("✅ Fungsi image_to_base64 siap digunakan.")

✅ Fungsi image_to_base64 siap digunakan.


In [8]:
# Tahap Bonus A.2: Fungsi Chat Multi-Modal (Teks + Gambar)
# Ini adalah versi modifikasi dari fungsi chat_with_ai sebelumnya

def chat_with_ai_multimodal(user_text_input, chat_history, image_base64_data_uri=None, model_name="gpt-4o"):
    """
    Mengirim input pengguna (teks dan/atau gambar base64) dan riwayat percakapan
    ke API OpenAI, dan mengembalikan respons dari AI.
    Riwayat chat_history diasumsikan hanya berisi pesan teks untuk kesederhanaan saat ini.
    """
    if not client:
        error_msg = "❌ Klien OpenAI tidak diinisialisasi. Fungsi chat multi-modal tidak dapat berjalan."
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_msg}</div>"))
        # Kembalikan format yang sama seperti sukses agar loop tidak error
        return error_msg, chat_history

    # --- Membangun Pesan Pengguna Multi-Modal ---
    user_message_content_parts = []
    # Selalu ada bagian teks
    user_message_content_parts.append({"type": "text", "text": user_text_input})

    if image_base64_data_uri:
        user_message_content_parts.append({
            "type": "image_url",
            "image_url": {"url": image_base64_data_uri}
        })
        # Tidak perlu display Markdown di sini karena akan ditampilkan di loop utama

    # Buat pesan pengguna baru dengan konten multi-modal
    new_user_message = {"role": "user", "content": user_message_content_parts}

    # Tambahkan pesan pengguna baru ke riwayat
    # Perhatian: Riwayat sebelumnya mungkin hanya teks.
    # Model seperti gpt-4o bisa menangani campuran ini, tapi ini adalah penyederhanaan.
    # Untuk sistem yang lebih canggih, Anda mungkin perlu memproses riwayat gambar juga.
    current_chat_history = chat_history + [new_user_message]

    try:
        # display(Markdown(f"DEBUG: Mengirim ke model {model_name} dengan pesan: {current_chat_history}"))
        response = client.chat.completions.create(
            model=model_name,
            messages=current_chat_history, # Kirim riwayat yang sudah diperbarui
            max_tokens=500 # Mungkin perlu lebih banyak token untuk deskripsi gambar
        )
        ai_reply = response.choices[0].message.content

        # Tambahkan balasan AI ke riwayat (balasan AI akan selalu teks)
        updated_chat_history = current_chat_history + [{"role": "assistant", "content": ai_reply}]

        return ai_reply, updated_chat_history # Kembalikan riwayat yang sudah termasuk pesan user & AI terakhir

    except openai.APIError as e:
        error_message = f"😥 Maaf, terjadi kesalahan API OpenAI: {e}"
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_message}</div>"))
        # Jangan tambahkan pesan error ke riwayat, kembalikan riwayat sebelum panggilan gagal
        return error_message, chat_history
    except Exception as e:
        error_message = f"🤯 Aduh, ada error tak terduga: {e}"
        display(Markdown(f"<div style='color: red; border: 1px solid red; padding: 10px; border-radius: 5px;'>{error_message}</div>"))
        return error_message, chat_history

print("✅ Fungsi chat_with_ai_multimodal siap digunakan.")

✅ Fungsi chat_with_ai_multimodal siap digunakan.


In [11]:
# Tahap Bonus B: Loop Interaktif Aplikasi Chat Multi-Modal

# --- PASTIKAN IMPOR ADA DI SINI ---
from google.colab import files
import mimetypes
import base64
# --- AKHIR BAGIAN IMPOR ---

if not client:
    display(Markdown("🔴 **Tidak dapat menjalankan chat:** Klien OpenAI belum siap. Pastikan kunci API sudah benar dan klien terinisialisasi."))
else:
    display(Markdown("🚀 **Aplikasi Chat Multi-Modal Siap Dimulai!** 🚀"))
    print("Ketik pesan Anda. Jika ingin menyertakan gambar, jawab 'y' saat ditanya.")
    print("Ketik 'exit' atau 'quit' untuk keluar.")
    print("-" * 50)

    chosen_model = "gpt-4o"
    display(Markdown(f"ℹ️ *Menggunakan model: **{chosen_model}***"))

    chat_history_multimodal = [
        {"role": "system", "content": "Kamu adalah AsistenAI multi-modal yang cerdas dan ramah. Kamu bisa memahami teks dan gambar. Berikan respons yang relevan berdasarkan input yang diberikan."}
    ]

    MAX_HISTORY_TURNS = 10

    while True:
        current_image_bytes = None
        current_image_mime_type = None
        image_data_uri_for_api = None # Reset untuk setiap giliran

        try:
            user_text_input = input("👤 Anda (teks): ")

            if user_text_input.lower() in ['exit', 'quit']:
                display(Markdown("👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!"))
                break

            # Cek apakah pengguna ingin mengunggah gambar
            ask_for_image = input("🖼️ Ingin menyertakan gambar dengan pesan ini? (y/n, default: n): ").lower().strip()

            if ask_for_image == 'y':
                display(Markdown("📂 **Silakan unggah satu file gambar (PNG, JPG, GIF, WEBP):**"))
                # Menggunakan try-except untuk files.upload() karena bisa di-cancel pengguna
                try:
                    uploaded_image_files = files.upload()
                    if uploaded_image_files:
                        uploaded_file_name = next(iter(uploaded_image_files))
                        current_image_bytes = uploaded_image_files[uploaded_file_name]

                        mime_type, _ = mimetypes.guess_type(uploaded_file_name)
                        if mime_type and mime_type.startswith("image/"):
                            current_image_mime_type = mime_type
                            display(Markdown(f"👍 Gambar '{uploaded_file_name}' (tipe: {current_image_mime_type}) berhasil diunggah."))
                        else:
                            current_image_mime_type = "image/png" # Default aman
                            display(Markdown(f"👍 Gambar '{uploaded_file_name}' berhasil diunggah. Tipe MIME tidak terdeteksi, menggunakan default '{current_image_mime_type}'."))

                        image_data_uri_for_api = image_to_base64(current_image_bytes, current_image_mime_type)
                    else:
                        display(Markdown("⚠️ Tidak ada gambar yang diunggah. Melanjutkan dengan teks saja."))
                except Exception as e_upload: # Menangkap error jika upload dibatalkan atau gagal
                     display(Markdown(f"ℹ️ Proses unggah gambar dibatalkan atau gagal: {e_upload}. Melanjutkan dengan teks saja."))
                     current_image_bytes = None # Pastikan ini None jika gagal
                     image_data_uri_for_api = None


            # Lanjutkan hanya jika ada teks atau gambar yang valid
            if not user_text_input.strip() and not image_data_uri_for_api:
                display_chat_bubble("assistant", "Hmm, sepertinya Anda belum mengetik apa-apa atau menyertakan gambar. Ada yang bisa saya bantu?", chosen_model)
                continue

            # Tampilkan pesan pengguna (hanya teksnya untuk display bubble standar)
            display_chat_bubble("user", user_text_input if user_text_input.strip() else "(Tidak ada input teks, hanya gambar)")
            if image_data_uri_for_api:
                 display(Markdown(f"🖼️ *(Gambar disertakan dalam permintaan ke AI)*"))

            ai_response, updated_history = chat_with_ai_multimodal(
                user_text_input,
                chat_history_multimodal,
                image_base64_data_uri=image_data_uri_for_api,
                model_name=chosen_model
            )
            chat_history_multimodal = updated_history

            display_chat_bubble("assistant", ai_response, chosen_model)

            if len(chat_history_multimodal) > (1 + MAX_HISTORY_TURNS * 2):
                display(Markdown(f"<i>(Riwayat percakapan dipangkas. Menyimpan {MAX_HISTORY_TURNS} interaksi terakhir...)</i>"))
                chat_history_multimodal = [chat_history_multimodal[0]] + chat_history_multimodal[-(MAX_HISTORY_TURNS*2):]

        except KeyboardInterrupt:
            display(Markdown("\n👋 Sesi chat dihentikan oleh pengguna. Sampai jumpa!"))
            break
        except EOFError:
            display(Markdown("\n👋 Input stream berakhir. Sampai jumpa!"))
            break
        except Exception as e:
            display(Markdown(f"🚨 **Terjadi error pada loop utama chat:** {str(e)}"))
            # Cetak traceback untuk debugging lebih lanjut jika errornya tidak jelas
            import traceback
            traceback.print_exc()
            # Pertimbangkan untuk break atau continue tergantung jenis error
            # continue

    print("-" * 50)
    display(Markdown("🏁 **Sesi Chat Multi-Modal Selesai.**"))

🚀 **Aplikasi Chat Multi-Modal Siap Dimulai!** 🚀

Ketik pesan Anda. Jika ingin menyertakan gambar, jawab 'y' saat ditanya.
Ketik 'exit' atau 'quit' untuk keluar.
--------------------------------------------------


ℹ️ *Menggunakan model: **gpt-4o***

👤 Anda (teks): tolong jelaskan gambar apa ini
🖼️ Ingin menyertakan gambar dengan pesan ini? (y/n, default: n): y


📂 **Silakan unggah satu file gambar (PNG, JPG, GIF, WEBP):**

...


    <div style="display: flex; justify-content: flex-end;">
        <div style="background-color: #FCE4EC; color: #880E4F; border-radius: 15px 15px 15px 0; padding: 10px 15px; margin: 5px 5px 5px 50px; max-width: 70%; float: right; clear: both; box-shadow: -2px 2px 5px rgba(0,0,0,0.1);">
            <strong style="font-size: 0.9em;">🤖 AI (gpt-4o):</strong><br>
            Monumen Nasional (Monas) dibangun untuk memperingati perjuangan rakyat Indonesia dalam mencapai kemerdekaan. Peletakan batu pertama dilakukan pada 17 Agustus 1961, di bawah pemerintahan Presiden Soekarno. Monas dirancang oleh arsitek Friedrich Silaban dan R.M. Soedarsono.<br><br>Monas memiliki tinggi 132 meter dan puncaknya dihiasi oleh lidah api berlapis emas. Monumen ini diresmikan pada 12 Juli 1975. Di dalamnya terdapat museum sejarah perjuangan kemerdekaan Indonesia, dan di bagian atasnya terdapat dek observasi yang menawarkan pemandangan Kota Jakarta.<br><br>Monas juga menjadi lokasi berbagai acara penting dan peringatan nasional. Monumen ini merupakan simbol semangat kemerdekaan dan kebanggaan nasional bagi rakyat Indonesia.
        </div>
    </div>
    <div style="clear: both;"></div>
    

👤 Anda (teks): exit


👋 Terima kasih sudah mengobrol! Sampai jumpa lagi!

--------------------------------------------------


🏁 **Sesi Chat Multi-Modal Selesai.**